
### PLOT GRAVEYARD:


In [ ]:
def plot_iad(
    ax,
    hipparcos_object=beetle_results.system.hipparcos_IAD,
    segment_halflength=3,
    ra_model=None,
    dec_model=None,
):
    """
    middle_ra and middle_dec are where to plot

    TODO: add IAD erorrs to this plot
    """

    cosphi = hipparcos_object.cos_phi
    sinphi = hipparcos_object.sin_phi
    if ra_model is None and dec_model is None:
        ra_model = np.zeros(len(cosphi))
        dec_model = np.zeros(len(cosphi))
    for i in range(len(hipparcos_object.alpha_abs_st)):
        ax.plot(
            np.array(
                [
                    hipparcos_object.alpha_abs_st[i] - segment_halflength * cosphi[i],
                    hipparcos_object.alpha_abs_st[i] + segment_halflength * cosphi[i],
                ]
            )
            - ra_model[i],
            np.array(
                [
                    hipparcos_object.delta_abs[i] - segment_halflength * sinphi[i],
                    hipparcos_object.delta_abs[i] + segment_halflength * sinphi[i],
                ]
            )
            - dec_model[i],
            color="black",
            alpha=0.5,
            # ls="",
            lw=1,
            marker="",
        )

    # get model prediction (ra, dec) at each hipparcos epoch
    # compute intersection of hipparcos scan with model prediction
    # plot intersection point plus/minus segment_length

In [ ]:
# make a plot with showing the star's total motion. Thinking a la Nowak+ beta Pic c paper figure.

# panel 1: full fit, radio + IAD + random draws from posterior
# panel 2: median PM subtracted, data points overplotted. uncertainty in data
# points (in lighter color) includes uncertainty in model subtraction
# panel 3: median PM + parallax motion subtracted, data points overplotted. uncertainty
#  in data points (in darker color) includes uncertainty in model subtraction


# TODO: add Hipparcos IAD to this figure
ra_data = beetle_results.system.data_table["quant1"]
dec_data = beetle_results.system.data_table["quant2"]
ra_err_data = beetle_results.system.data_table["quant1_err"]
dec_err_data = beetle_results.system.data_table["quant2_err"]
epochs_data = beetle_results.system.data_table["epoch"]

fig, ax = plt.subplots(3, 2, figsize=(30, 8), sharex=True)
plt.subplots_adjust(hspace=0)

# pick some random orbits from the posterior
num2plot = 50

restricted_post = beetle_results.post[
    (beetle_results.post[:, beetle_results.system.param_idx["per1"]] > 1900 / 365.25)
    & (beetle_results.post[:, beetle_results.system.param_idx["per1"]] < 2600 / 365.25)
]

plot_indices = np.random.randint(0, len(restricted_post), size=num2plot)

epochs2plot = np.linspace(
    np.min(beetle_results.system.data_table["epoch"]),
    np.max(beetle_results.system.data_table["epoch"]),
    int(1e3),
)

raoff_orbit, deoff_orbit, _ = beetle_results.system.compute_all_orbits(
    restricted_post[plot_indices].T,
    epochs2plot,
)
raoff_star = raoff_orbit[:, 0, :]
deoff_star = deoff_orbit[:, 0, :]

whole_racosdelta_model = np.empty((num2plot, len(epochs2plot)))
whole_dec_model = np.empty((num2plot, len(epochs2plot)))
cosdelta0 = np.cos(np.radians(beetle_results.system.hipparcos_IAD.delta0))
pmra_residuals = np.empty((num2plot, len(epochs_data)))
pmdec_residuals = np.empty((num2plot, len(epochs_data)))
pmplx_ra_residuals = np.empty((num2plot, len(epochs_data)))
pmplx_dec_residuals = np.empty((num2plot, len(epochs_data)))

for i in np.arange(num2plot):
    (
        raoff_cosdelta0_pmplx,
        deoff_pmplx,
    ) = beetle_results.system.pm_plx_predictor.compute_astrometric_model(
        beetle_results.post[plot_indices[i]],
        beetle_results.system.param_idx,
        epochs=Time(epochs2plot, format="mjd").decimalyear,
    )
    whole_racosdelta_model[i, :] = raoff_cosdelta0_pmplx + raoff_star[:, i] * cosdelta0
    whole_dec_model[i, :] = deoff_pmplx + deoff_star[:, i]
    ax[0, 0].plot(epochs2plot, whole_racosdelta_model[i, :], color="grey", alpha=0.1)
    ax[0, 1].plot(epochs2plot, whole_dec_model[i, :], color="grey", alpha=0.1)

    # plot model with PM subtracted
    pm_ra = beetle_results.post[plot_indices[i]][
        beetle_results.system.param_idx["pm_ra"]
    ]
    pm_racosdelta_prediction = (
        Time(epochs2plot, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_ra

    ax[1, 0].plot(
        epochs2plot,
        whole_racosdelta_model[i, :] - pm_racosdelta_prediction,
        color="grey",
        alpha=0.1,
    )

    pm_dec = beetle_results.post[plot_indices[i]][
        beetle_results.system.param_idx["pm_dec"]
    ]
    pm_dec_prediction = (
        Time(epochs2plot, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_dec

    ax[1, 1].plot(
        epochs2plot,
        whole_dec_model[i, :] - pm_dec_prediction,
        color="grey",
        alpha=0.1,
    )

    # compute residuals to PM only
    pm_racosdelta_prediction_fordata = (
        Time(epochs_data, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_ra
    pm_dec_prediction_fordata = (
        Time(epochs_data, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_dec

    pmdec_residuals[i, :] = dec_data - pm_dec_prediction_fordata
    pmra_residuals[i, :] = ra_data * cosdelta0 - pm_racosdelta_prediction_fordata

    # compute residuals to PM + plx
    post = beetle_results.post[plot_indices[i], :]
    post_zeromass = np.copy(post)
    post_zeromass[beetle_results.system.param_idx["m1"]] = 0

    pmplx_prediction_fordata = beetle_results.system.compute_model(post_zeromass)

    pmplx_ra_residuals[i, :] = ra_data * cosdelta0 - pmplx_prediction_fordata[0][:, 0]
    pmplx_dec_residuals[i, :] = dec_data - pmplx_prediction_fordata[0][:, 1]

    # plot only orbital motion of model
    ax[2, 0].plot(epochs2plot, raoff_star[:, i] * cosdelta0, color="grey", alpha=0.1)
    ax[2, 1].plot(epochs2plot, deoff_star[:, i], color="grey", alpha=0.1)


# have: scan angle (self.cosphi/self.sinphi), abscissca(self.alpha_abs_st), time, unc (self.epc)


ax[0, 0].errorbar(
    epochs_data,
    ra_data * cosdelta0,
    yerr=ra_err_data,
    ls="",
    color="purple",
    marker="o",
)
ax[0, 1].errorbar(
    epochs_data,
    dec_data,
    yerr=dec_err_data,
    ls="",
    color="purple",
    marker="o",
)

# for each orbit, subtract the proper motion. Overplot obs errors in one color and std of subtraction in another
ax[1, 0].errorbar(
    epochs_data,
    np.median(pmra_residuals, axis=0),
    yerr=ra_err_data,
    color="purple",
    ls="",
    alpha=0.25,
)
ax[1, 0].errorbar(
    epochs_data,
    np.median(pmra_residuals, axis=0),
    yerr=np.std(pmra_residuals, axis=0),
    color="purple",
    ls="",
    marker="o",
)

ax[1, 1].errorbar(
    epochs_data,
    np.median(pmdec_residuals, axis=0),
    yerr=dec_err_data,
    color="purple",
    ls="",
    alpha=0.25,
)
ax[1, 1].errorbar(
    epochs_data,
    np.median(pmdec_residuals, axis=0),
    yerr=np.std(pmdec_residuals, axis=0),
    color="purple",
    ls="",
    marker="o",
)

# for each orbit, subtract the proper motion and parallax motion. Overplot obs errors in one color and std of subtraction in another
ax[2, 0].errorbar(
    epochs_data,
    np.median(pmplx_ra_residuals, axis=0),
    yerr=ra_err_data,
    color="purple",
    ls="",
    alpha=0.25,
)
ax[2, 0].errorbar(
    epochs_data,
    np.median(pmplx_ra_residuals, axis=0),
    yerr=np.std(pmplx_ra_residuals, axis=0),
    color="purple",
    ls="",
    marker="o",
)

ax[2, 1].errorbar(
    epochs_data,
    np.median(pmplx_dec_residuals, axis=0),
    yerr=dec_err_data,
    color="purple",
    ls="",
    alpha=0.25,
)
ax[2, 1].errorbar(
    epochs_data,
    np.median(pmplx_dec_residuals, axis=0),
    yerr=np.std(pmplx_dec_residuals, axis=0),
    color="purple",
    ls="",
    marker="o",
)

ax[2, 0].set_xlabel("epoch [mjd]")
ax[2, 1].set_xlabel("epoch [mjd]")
ax[1, 0].set_ylabel("$\\Delta$R.A. $\\cos{\\delta_0}$ [mas]")
ax[1, 1].set_ylabel("$\\Delta$decl. [mas]")
# ax[1, 0].set_ylim(-10, 10)

plt.savefig("{}/multipanel_plot.png".format(savedir), dpi=250)

In [ ]:
import orbitize.results
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np

# choose which fit to inspect
fit_planet = False
dvd = False
burn_steps = 100
total_steps = 500000
renorm_hip = True


run_name = "planet{}_dvd{}_renormHIP{}_burn{}_total{}".format(
    fit_planet, dvd, renorm_hip, burn_steps, total_steps
)


# read in results
beetle_results = (
    orbitize.results.Results()
)  # create a blank results object to load the data
beetle_results.load_results("results/{}.hdf5".format(run_name))

harper17_solution = {  # table 3, 2.4 mas radio noise
    "alpha0": [0.09, 0.19],
    "delta0": [1.4, 0.48],
    "plx": [3.77, 2.20],
    "pm_ra": [25.53, 0.31],
    "pm_dec": [9.37, 0.28],
}

fig, ax = plt.subplots(5, 1, figsize=(5, 11))
for i, a in enumerate(ax):
    idx = 6 + i
    param = beetle_results.labels[idx]

    if param == "pm_ra":
        a.hist(
            beetle_results.post[:, idx] / np.cos(np.radians(7.40703653)),
            bins=50,
            color="grey",
            density=True,
            label="orbitize! refit",
        )
    else:
        a.hist(
            beetle_results.post[:, idx],
            bins=50,
            color="grey",
            density=True,
            label="orbitize! refit",
        )

    a.set_xlabel(param)
    if param not in ["alpha0", "delta0"]:
        xs = np.linspace(
            harper17_solution[param][0] - 5 * harper17_solution[param][1],
            harper17_solution[param][0] + 5 * harper17_solution[param][1],
        )
        a.plot(
            xs,
            norm(harper17_solution[param][0], harper17_solution[param][1]).pdf(xs),
            color="k",
            label="Harper+ 17 table 4, $\sigma_{{radio}}$ = 2.4mas",
        )

ax[3].set_xlabel("$\Delta\\alpha$cos($\delta$) [mas]")
ax[4].set_xlabel("$\Delta\\delta$ [mas]")


# # plx vs pm_ra
# ax[3].hist2d(beetle_results.post[:, 6], beetle_results.post[:, 7], bins=30)
# ax[3].set_xlabel(beetle_results.labels[6])
# ax[3].set_ylabel(beetle_results.labels[7])

# # plx vs pm_dec
# ax[4].hist2d(beetle_results.post[:, 6], beetle_results.post[:, 8], bins=30)
# ax[4].set_xlabel(beetle_results.labels[6])
# ax[4].set_ylabel(beetle_results.labels[8])

ax[0].legend()

plt.tight_layout()
plt.savefig("plots/{}.png".format(run_name))

In [ ]:
Compute the flux ratio & separation:
from astropy.modeling.physical_models import BlackBody

# ref: Levesque+ 2020 fig2 (est. by eye at 6800 Angstroms)
betelgeuse_flux = 10 ** (-8.5) * u.erg / u.s / u.cm**2 / u.angstrom

# assumptions of companion properties based on Morgan's guesses
assumed_params = {"distance": 200 * u.pc, "teff": 7500 * u.K, "radius": 0.8 * u.R_sun}

bb = BlackBody(
    assumed_params["teff"], scale=1 * u.erg / u.s / u.cm**2 / u.angstrom / u.sr
)


def companion_flux(wavelen):
    flux = (
        bb(wavelen)
        * np.pi
        * u.sr
        * (assumed_params["radius"]) ** 2
        / (assumed_params["dist"]) ** 2
    )
    return flux.to(u.erg / u.s / u.cm**2 / u.AA)


total_mass_post = (
    beetle_results.post[:, beetle_results.param_idx["m0"]]
    + beetle_results.post[:, beetle_results.param_idx["m1"]]
)


sma_post_au = ((per_post**2) * total_mass_post) ** (1 / 3)
sma_post_solrad = (sma_post_au * u.au).to(u.R_sun).value
dist_post = 1e3 / beetle_results.post[:, beetle_results.param_idx["plx"]]

sep_post = (sma_post_au * u.au / (dist_post * u.pc)).to(
    u.mas, equivalencies=u.dimensionless_angles()
)

plt.figure()
plt.hist(sep_post, bins=50, density=True)
plt.xlabel("sep [mas]")
plt.savefig("{}/separation.png".format(savedir), dpi=250)

Visualize the fit:

In [ ]:
"""
VERSION 2
"""

# make a plot with showing the star's total motion. Thinking a la Nowak+ beta Pic c paper figure.

# panel 1: full fit, radio + IAD + random draws from posterior
# panel 2: median PM subtracted, data points overplotted. uncertainty in data
# points (in lighter color) includes uncertainty in model subtraction
# panel 3: median PM + parallax motion subtracted, data points overplotted. uncertainty
#  in data points (in darker color) includes uncertainty in model subtraction

ra_data = beetle_results.system.data_table["quant1"]
dec_data = beetle_results.system.data_table["quant2"]
ra_err_data = beetle_results.system.data_table["quant1_err"]
dec_err_data = beetle_results.system.data_table["quant2_err"]
epochs_data = beetle_results.system.data_table["epoch"]

fig, ax = plt.subplots(3, 1, figsize=(8, 24))

# pick some random orbits from the posterior
num2plot = 50

restricted_post = beetle_results.post[
    (beetle_results.post[:, beetle_results.system.param_idx["per1"]] > 1900 / 365.25)
    & (beetle_results.post[:, beetle_results.system.param_idx["per1"]] < 2600 / 365.25)
]

plot_indices = np.random.randint(0, len(restricted_post), size=num2plot)

epochs2plot = np.linspace(
    np.min(beetle_results.system.data_table["epoch"]) - 100,
    np.max(beetle_results.system.data_table["epoch"]) + 1000,
    int(1e3),
)

raoff_orbit, deoff_orbit, _ = beetle_results.system.compute_all_orbits(
    restricted_post[plot_indices].T,
    epochs2plot,
)
raoff_star = raoff_orbit[:, 0, :]
deoff_star = deoff_orbit[:, 0, :]

whole_racosdelta_model = np.empty((num2plot, len(epochs2plot)))
whole_dec_model = np.empty((num2plot, len(epochs2plot)))
cosdelta0 = np.cos(np.radians(beetle_results.system.hipparcos_IAD.delta0))
pmra_residuals = np.empty((num2plot, len(epochs_data)))
pmdec_residuals = np.empty((num2plot, len(epochs_data)))
pmplx_ra_residuals = np.empty((num2plot, len(epochs_data)))
pmplx_dec_residuals = np.empty((num2plot, len(epochs_data)))

for i in np.arange(num2plot):
    (
        raoff_cosdelta0_pmplx,
        deoff_pmplx,
    ) = beetle_results.system.pm_plx_predictor.compute_astrometric_model(
        beetle_results.post[plot_indices[i]],
        beetle_results.system.param_idx,
        epochs=Time(epochs2plot, format="mjd").decimalyear,
    )
    whole_racosdelta_model[i, :] = raoff_cosdelta0_pmplx + raoff_star[:, i] * cosdelta0
    whole_dec_model[i, :] = deoff_pmplx + deoff_star[:, i]
    ax[0].plot(
        whole_racosdelta_model[i, :],
        whole_dec_model[i, :],
        color="grey",
        alpha=0.1,
    )

    # plot model with PM subtracted
    pm_ra = beetle_results.post[plot_indices[i]][
        beetle_results.system.param_idx["pm_ra"]
    ]
    pm_racosdelta_prediction = (
        Time(epochs2plot, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_ra

    pm_dec = beetle_results.post[plot_indices[i]][
        beetle_results.system.param_idx["pm_dec"]
    ]
    pm_dec_prediction = (
        Time(epochs2plot, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_dec

    ax[1].plot(
        whole_racosdelta_model[i, :] - pm_racosdelta_prediction,
        whole_dec_model[i, :] - pm_dec_prediction,
        color="grey",
        alpha=0.1,
    )

    # compute residuals to PM only
    pm_racosdelta_prediction_fordata = (
        Time(epochs_data, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_ra
    pm_dec_prediction_fordata = (
        Time(epochs_data, format="mjd").decimalyear
        - beetle_results.system.hipparcos_IAD.alphadec0_epoch
    ) * pm_dec

    pmdec_residuals[i, :] = dec_data - pm_dec_prediction_fordata
    pmra_residuals[i, :] = ra_data * cosdelta0 - pm_racosdelta_prediction_fordata

    # compute residuals to PM + plx
    post = beetle_results.post[plot_indices[i], :]
    post_zeromass = np.copy(post)
    post_zeromass[beetle_results.system.param_idx["m1"]] = 0

    pmplx_prediction_fordata = beetle_results.system.compute_model(post_zeromass)

    pmplx_ra_residuals[i, :] = ra_data - pmplx_prediction_fordata[0][:, 0]
    pmplx_dec_residuals[i, :] = dec_data - pmplx_prediction_fordata[0][:, 1]

    # plot only orbital motion of model
    ax[2].plot(
        raoff_star[:, i] * cosdelta0,
        deoff_star[:, i],
        color="grey",
        alpha=0.1,
    )


# plot IAD
plot_iad(ax[0])


# for each orbit, subtract the proper motion. Overplot obs errors in one color and std of subtraction in another
ax[0].errorbar(
    ra_data * cosdelta0,
    dec_data,
    xerr=ra_err_data,
    yerr=dec_err_data,
    ls="",
    color="purple",
    marker="o",
)

pm_racosdelta_prediction_forIAD = (
    Time(beetle_results.system.hipparcos_IAD.epochs_mjd, format="mjd").decimalyear
    - beetle_results.system.hipparcos_IAD.alphadec0_epoch
) * pm_ra
pm_dec_prediction_forIAD = (
    Time(beetle_results.system.hipparcos_IAD.epochs_mjd, format="mjd").decimalyear
    - beetle_results.system.hipparcos_IAD.alphadec0_epoch
) * pm_dec


plot_iad(
    ax[1], ra_model=pm_racosdelta_prediction_forIAD, dec_model=pm_dec_prediction_forIAD
)


# for each orbit, subtract the proper motion and parallax motion. Overplot obs errors in one color and std of subtraction in another
ax[1].errorbar(
    np.median(pmra_residuals, axis=0),
    np.median(pmdec_residuals, axis=0),
    xerr=ra_err_data,
    yerr=dec_err_data,
    color="purple",
    ls="",
    alpha=0.25,
    marker="o",
)
ax[2].errorbar(
    np.median(pmplx_ra_residuals, axis=0),
    np.median(pmplx_dec_residuals, axis=0),
    xerr=ra_err_data,
    yerr=dec_err_data,
    color="purple",
    ls="",
    alpha=0.25,
    marker="o",
)

racosdelta0_pmplx_IAD = np.zeros(
    (num2plot, len(beetle_results.system.hipparcos_IAD.epochs_mjd))
)
dec_pmplx_IAD = np.zeros(
    (num2plot, len(beetle_results.system.hipparcos_IAD.epochs_mjd))
)
for i in range(num2plot):
    ra, dec = beetle_results.system.pm_plx_predictor.compute_astrometric_model(
        restricted_post[i],
        beetle_results.system.param_idx,
        epochs=beetle_results.system.hipparcos_IAD.epochs,
    )

    racosdelta0_pmplx_IAD[i, :] = ra
    dec_pmplx_IAD[i, :] = dec

# TODO: the below is wrong; need to fix residual calculation.
plot_iad(
    ax[2],
    ra_model=np.median(racosdelta0_pmplx_IAD, axis=0),
    dec_model=np.median(dec_pmplx_IAD, axis=0),
    segment_halflength=1,
)


# ax[2, 0].errorbar(
#     epochs_data,
#     np.median(pmplx_ra_residuals, axis=0),
#     yerr=np.std(pmplx_ra_residuals, axis=0),
#     color="purple",
#     ls="",
#     marker="o",
# )

# ax[2, 1].errorbar(
#     epochs_data,
#     np.median(pmplx_dec_residuals, axis=0),
#     yerr=dec_err_data,
#     color="purple",
#     ls="",
#     alpha=0.25,
# )
# ax[2, 1].errorbar(
#     epochs_data,
#     np.median(pmplx_dec_residuals, axis=0),
#     yerr=np.std(pmplx_dec_residuals, axis=0),
#     color="purple",
#     ls="",
#     marker="o",
# )

# ax[2, 0].set_xlabel("epoch [mjd]")
# ax[2, 1].set_xlabel("epoch [mjd]")
for a in ax:
    a.set_xlabel("$\\Delta$R.A. $\\cos{\\delta_0}$ [mas]")
    a.set_ylabel("$\\Delta$decl. [mas]")
# ax[1, 0].set_ylim(-10, 10)


ax[0].set_ylim(-50, 25)
ax[0].set_xlim(-100, 100)

ax[1].set_ylim(-20, 20)
ax[1].set_xlim(-20, 20)

ax[2].set_xlim(-5, 5)
ax[2].set_ylim(-5, 5)


ax[0].set_title("full fit (pm + plx + orbit motion)")
ax[1].set_title("plx + orbit motion")
ax[2].set_title("orbit motion")

plt.savefig("{}/multipanel_plot.png".format(savedir), dpi=250)

Compare orbital phase of RV and astrometric orbits:

In [ ]:
from orbitize.basis import tau_to_tp
from radvel.basis import timeperi_to_timetrans


tau_post = beetle_results.post[:, beetle_results.param_idx["tau1"]]
per_post = beetle_results.post[:, beetle_results.param_idx["per1"]]
ecc_post = beetle_results.post[:, beetle_results.param_idx["ecc1"]]
aop_pl_post = beetle_results.post[:, beetle_results.param_idx["aop1"]]
pan_post = beetle_results.post[:, beetle_results.param_idx["pan1"]]
aop_st_post = aop_pl_post + np.pi
per_post_days = per_post * 365.25

match_per_mask = (per_post_days > 2000) & (per_post_days < 2500)

tP_post = tau_to_tp(tau_post, beetle_results.tau_ref_epoch, per_post)
tC_post = timeperi_to_timetrans(tP_post, per_post_days, ecc_post, aop_st_post)


after_date = after_date = Time(2018, format="decimalyear").mjd
num_periods = (after_date - tC_post) / per_post_days
num_periods = np.ceil(num_periods).astype(int)
tC_post += num_periods * per_post_days

fig, ax = plt.subplots(1, 1)
ax.hist(
    Time(tC_post, format="mjd").decimalyear,
    bins=50,
    color="rebeccapurple",
    label="all P",
    density=True,
)
# ax.hist(
#     Time(tC_post[match_per_mask], format="mjd").decimalyear,
#     bins=50,
#     color="pink",
#     histtype="step",
#     label="2000 d < P < 2500 d",
# )
ax.hist(
    Time(
        tC_post[match_per_mask & (np.degrees(pan_post) > 150)], format="mjd"
    ).decimalyear,
    bins=50,
    color="hotpink",
    histtype="step",
    label="2000 d < P < 2500 d AND $\\Omega_{{*}}$ ~ 50deg",
    density=True,
)
ax.legend()
ax.set_xlabel("first time of conj. after 2018 [yr]")
plt.savefig("{}/phase.png".format(savedir), dpi=250)

ModuleNotFoundError: No module named 'radvel'

In [ ]:

Compute amplitude of astrometric signal & goodness-of-fit:
# calculate magnitude of astrometric signal over a single orbital period

# use calc_orbit to get relative ra/dec over epochs spanning 4000 d, multiply by m1/m0 to get mag of signal

epochs = np.linspace(0, 4000, int(1e3))

# for each orbit in a randomly selected 100 orbit subset of the posterior, solve the orbit
num2pick = 75
plot_indices = np.random.randint(0, len(beetle_results.post), size=num2pick)

raoff_orbit, deoff_orbit, _ = beetle_results.system.compute_all_orbits(
    beetle_results.post[plot_indices].T,
    epochs,
)
m0_posterior2plot = beetle_results.post[:, beetle_results.system.param_idx["m0"]][
    plot_indices
]
m1_posterior2plot = beetle_results.post[:, beetle_results.system.param_idx["m1"]][
    plot_indices
]

max_seps = np.empty(num2pick)
med_seps = np.empty(num2pick)
fig, ax = plt.subplots(1, 1)
for i in range(num2pick):

    plt.plot(
        raoff_orbit[:, 1, i],
        deoff_orbit[:, 1, i],
        alpha=0.1,
        lw=1,
        color="grey",
    )

    separations = (
        np.sqrt(raoff_orbit[:, 1, i] ** 2 + deoff_orbit[:, 1, i] ** 2)
        * m1_posterior2plot[i]
        / m0_posterior2plot[i]
    )
    max_seps[i] = np.max(separations)
    med_seps[i] = np.median(separations)

plt.plot([0], [0], marker="*", color="rebeccapurple")

plt.ylabel("$\Delta$decl. [mas]")
plt.xlabel("$\Delta$RA [mas]")
plt.savefig("{}/orbit_plot.png".format(savedir), dpi=250)

plt.figure()
plt.hist(max_seps, bins=50, label="max distance from barycenter", alpha=0.5)
plt.hist(med_seps, bins=50, label="median distance from barycenter", alpha=0.5)
plt.legend()
plt.xlabel("betelgeuse A separation [mas]")
plt.savefig("{}/astrom_signal_size.png".format(savedir), dpi=250)